In [16]:
from nba_api.stats.endpoints import teamgamelogs
import pandas as pd
import datetime as dt

In [22]:
team = teamgamelogs.TeamGameLogs(season_nullable='2021-22').get_data_frames()[0]
team.columns = [col.lower() for col in team.columns]
team['game_id'] = team['game_id'].str[2:]
team = team.drop_duplicates(subset=['game_id'])

# this will be useful later on
# team_abbr = {
#     'MEM':'Memphis Grizzlies',
#     'IND':'Indiana Pacers',
#     'ORL':'Orlando Magic',
#     'DEN':'Denver Nuggets',
#     'LAL':'Los Angeles Lakers',
#     'TOR':'Toronto Raptors',
#     'CHI':'Chicago Bulls',
#     'SAC':'Sacramento Kings',
#     'BKN':'Brooklyn Nets',
#     'DET':'Detroit Pistons',
#     'PHI':'Philadelphia 76ers',
#     'MIL':'Milwaukee Bucks',
#     'POR':'Portland Trail Blazers',
#     'NYK':'New York Knicks',
#     'MIA':'Miami Heat',
#     'BOS':'Boston Celtics',
#     'UTA':'Utah Jazz',
#     'GSWL':'Golden State Warriors',
#     'SAS':'San Antonio Spurs',
#     'DAL':'Dallas Mavericks',
#     'LAC':'LA Clippers',
#     'CLE':'Cleveland Cavaliers',
#     'WAS':'Washington Wizards',
#     'MIN':'Minnesota Timberwolves',
#     'HOU':'Houston Rockets',
#     'PHX':'Phoenix Suns',
#     'OKC':'Oklahoma City Thunder',
#     'CHAR':'Charlotte Hornets',
#     'ATL':'Atlanta Hawks',
#     'NOP':'New Orleans Pelicans'
# }

In [23]:
# read data and prep columns/data types
df = pd.read_csv('test_data/2021_game_logs.csv')
df = df.drop(['season_id','video_available','game_date'],axis='columns')
df['game_id'] = df['game_id'].astype(str)

In [24]:
# calculate fantasy scores for each player game log
df['fantasy_score'] = (df.fg3m*3 +
                      df.fgm*2 + 
                      ((df.fg3a + df.fga) - (df.fg3m + df.fgm))*-1 + 
                      df.ftm*1 +
                      (df.fta - df.ftm)*-0.5 +
                      df.dreb*1.2 +
                      df.oreb*1.5 +
                      df.ast*1.5 +
                      df.blk*2 + 
                      df.stl*2 +
                      df.tov*-1)
game_log_cols = df.columns

In [25]:
# get sortable game_dates from team game log
df = df.merge(team, how='left', on='game_id', suffixes=['','_t'])
df = df.drop([col for col in df if col != 'game_date' and col not in game_log_cols], axis='columns')
df['game_date'] = pd.to_datetime(df['game_date'])
df = df.sort_values(by='game_date').reset_index(drop=True)
df.insert(4,'game_date', df.pop('game_date'))

In [26]:
# calculate fantasy score cumulative sum and merge back onto original df
df_cumul = (df[['player_id','player_name','game_id','game_date','fantasy_score']]
                .groupby(['player_id','player_name','game_id','game_date']).sum()\
                .groupby(level=0).cumsum()
                .reset_index())
df = (df.merge(df_cumul,how='left',on=['player_id','player_name','game_id','game_date'],suffixes=['','_cumul'])
       .sort_values(by=['player_id','game_date'])
       .reset_index(drop=True))

In [27]:
# save to csv
df.to_csv('test_data/2021_fantasy_scores.csv',index=False)